In [ ]:
    L, K = calculate_periodic_points(f,g)
    if len(L) != len(K):  # checks maps have the same number of fixed points
        return []
    mult_to_point_L = {}
    mult_to_point_K = {}
    point_to_mult_L = {}
    point_to_mult_K = {}
    level = 0
    initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    repeated_mult_L = {}
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            pass
        elif len(mult_to_point_K[mult_L]) != repeated:
            pass
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]
    #print('repeated_mult:', repeated_mult_L)
    d = len(L)
    r = f.domain().base_ring()
    more = True
    source = []
    corresponding = []
    more = find_independent_set(repeated_mult_L, source, corresponding)
    #print('corresponding:', corresponding)
    i_repeated_mult = deepcopy(repeated_mult_L)
    a_repeated_mult = {}
    found_no_more = True
    while more:
        level += 1
        more, found_no_more, source, corresponding = (i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult)
        if found_no_more:
            raise ValueError('no more rational preimages')
        i_repeated_mult = deepcopy(a_repeated_mult)
        a_repeated_mult = {}
        found_no_more = True
        #print('i_repeated_mult:', i_repeated_mult)
    #print('source:', source)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    possible_targets = []
    arrangements_iterators = []
    initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators)
    Conj = []
    find_conjugations(arrangements_iterators, possible_targets, source, Conj)
    return Conj

In [1]:
from itertools import product
from copy import deepcopy

In [2]:
def isLinearlyIndependent(S):
    M = matrix([list(t) for t in S])
    if len(S) < n+1:
        if M.rank() == len(S):
            return True
        return False
    else:
        if any(l == 0 for l in M.minors(n+1)):
            return False
        return True

In [3]:
def calculate_periodic_points(f, g):
    return Set(f.periodic_points(1)), Set(g.periodic_points(1))

In [13]:
def initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L):
    for i in range(len(L)):
        mult_L = f.multiplier(L[i], 1).charpoly()
        mult_K = g.multiplier(K[i], 1).charpoly()
        tup_L = (mult_L, level)
        tup_K = (mult_K, level)
        if tup_L not in mult_to_point_L:
            mult_to_point_L[tup_L] = [L[i]]
        else:
            mult_to_point_L[tup_L] += [L[i]]
        if tup_K not in mult_to_point_K:
            mult_to_point_K[tup_K] = [K[i]]
        else:
            mult_to_point_K[tup_K] += [K[i]]
        point_to_mult_L[L[i]] = (mult_L, 0)
        point_to_mult_K[K[i]] = (mult_K, 0)
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            pass
        elif len(mult_to_point_K[mult_L]) != repeated:
            pass
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]

In [5]:
def find_independent_set(repeated_mult_L, source, corresponding):
    more = True
    for r in repeated_mult_L:
        for point_lst in repeated_mult_L[r]:
            for point in point_lst:
                if isLinearlyIndependent(source + [point]):
                    source.append(point)
                    mult = point_to_mult_L[point]
                    # if another point with this multiplier and level pair is in S
                    # then the multiplier level pair will be the last element of corresponding
                    if len(corresponding) != 0:
                        if corresponding[-1][0] == mult:
                            corresponding[-1][1] += 1
                        else:
                            corresponding.append([mult, 1])
                    else:
                        corresponding.append([mult, 1])
                if len(source) == n+2:
                    more = False
                    break
            if len(source) == n+2:
                break
        if len(source) == n+2:
            break
    return more

In [6]:
def find_preimages(i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult):
    for r in i_repeated_mult:
            for point_lst_L in i_repeated_mult[r]:
                old_tup_L = point_to_mult_L[point_lst_L[0]]
                #print('current tup:', old_tup_L)
                point_lst_K = mult_to_point_K[old_tup_L]
                mult_L = old_tup_L[0]
                Tl = []
                Tk = []
                for pnt in point_lst_L:
                    for preimage in f.rational_preimages(pnt):
                        if preimage != pnt:
                            Tl.append(preimage)
                for pnt in point_lst_K:
                    for preimage in g.rational_preimages(pnt):
                        if preimage != pnt:
                            Tk.append(preimage)
                #print('Tl:', Tl)
                #print('Tk:', Tk)
                if len(Tl) != len(Tk):
                    return []
                if len(Tl) != 0:
                    found_no_more = False
                    new_tup_L = (mult_L, level)
                    new_tup_K = (mult_L, level)
                    #print('new_tup_L:', new_tup_L)
                    mult_to_point_L[new_tup_L] = Tl
                    mult_to_point_K[new_tup_K] = Tk
                    for i in range(len(Tl)):
                        point_to_mult_L[Tl[i]] = new_tup_L
                        point_to_mult_K[Tk[i]] = new_tup_K
                    repeated = len(Tl)
                    if repeated not in repeated_mult_L:
                        repeated_mult_L[repeated] = [Tl]
                    else:
                        repeated_mult_L[repeated] += [Tl]
                    if repeated not in a_repeated_mult:
                        a_repeated_mult[repeated] = [Tl]
                    else:
                        a_repeated_mult[repeated] += [Tl]
                    # right now rechecking independence
                    source = []
                    corresponding = []
                    more = find_independent_set(repeated_mult_L, source, corresponding)
                if not more:
                    break
            if not more:
                break
    return more, found_no_more, source, corresponding

In [7]:
def initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators):
    for tup in corresponding:
        possible_targets.append([mult_to_point_K[tup[0]], tup[1]])
    #print('possible_targets:', possible_targets)
    for lst in possible_targets:
        arrangements_iterators.append(Arrangements(range(len(lst[0])), lst[1]))

In [8]:
def find_conjugations(arrangements_iterators, possible_targets, source, Conj, f, g):
    for tup in product(*arrangements_iterators):
        current_target = []
        for i in range(len(tup)):
            for j in tup[i]:
                current_target.append(possible_targets[i][0][j])
        #print('current_target:', current_target)
        try:
            phi = f.domain().point_transformation_matrix(current_target, source)
            #print('phi:', phi)
            #print('phi inverse', phi.inverse())
            #print([phi*pnt for pnt in current_target])
            #%time f.conjugate(phi) == g
            if f.conjugate(phi) == g:
                Conj.append(phi)
        except ValueError:
            pass

In [9]:
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])
g = f

In [10]:
L, K = calculate_periodic_points(f,g)

In [14]:
mult_to_point_L = {}
mult_to_point_K = {}
point_to_mult_L = {}
point_to_mult_K = {}
repeated_mult_L = {}
level = 0
initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L)

In [16]:
source = []
corresponding = []
n = f.domain().dimension_relative()
more = find_independent_set(repeated_mult_L, source, corresponding)

In [40]:
i_repeated_mult = deepcopy(repeated_mult_L)
a_repeated_mult = {}
found_no_more = True
while more:
    level += 1
    more, found_no_more, source, corresponding = (i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult)
    if found_no_more:
        raise ValueError('no more rational preimages')
    i_repeated_mult = deepcopy(a_repeated_mult)
    a_repeated_mult = {}
    found_no_more = True

In [19]:
possible_targets = []
arrangements_iterators = []
initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators)

In [56]:
%%time
Conj = []
find_conjugations(arrangements_iterators, possible_targets, source, Conj, f, g)
print(Conj)

[[1 0 0]
[0 1 0]
[0 0 1], [0 0 1]
[0 1 0]
[1 0 0], [1 0 0]
[0 0 1]
[0 1 0], [0 1 0]
[0 0 1]
[1 0 0], [0 1 0]
[1 0 0]
[0 0 1], [0 0 1]
[1 0 0]
[0 1 0]]


In [44]:
X = p_iter_fork(13)

In [57]:
%%time
conj = []
for ret in X(test_conj, parallel_data):
    if ret[1]:
        conj.append(ret[1])
print(conj)

[[1 0 0]
[0 1 0]
[0 0 1], [0 0 1]
[0 1 0]
[1 0 0], [1 0 0]
[0 0 1]
[0 1 0], [0 1 0]
[0 0 1]
[1 0 0], [0 1 0]
[1 0 0]
[0 0 1], [0 0 1]
[1 0 0]
[0 1 0]]


In [20]:
t = list(product(*arrangements_iterators))

In [22]:
from sage.parallel.use_fork import p_iter_fork

In [39]:
def test_conj(tup, possible_targets, f, g, source):
    current_target = []
    for i in range(len(tup)):
        for j in tup[i]:
            current_target.append(possible_targets[i][0][j])
    try:
        phi = f.domain().point_transformation_matrix(current_target, source)
        if f.conjugate(phi) == g:
            return phi
    except ValueError:
        pass
    return None

In [29]:
conj

[
[1 0 0]  [0 0 1]  [1 0 0]  [0 1 0]  [0 1 0]  [0 0 1]
[0 1 0]  [0 1 0]  [0 0 1]  [0 0 1]  [1 0 0]  [1 0 0]
[0 0 1], [1 0 0], [0 1 0], [1 0 0], [0 0 1], [0 1 0]
]

In [40]:
parallel_data = []
for tup in t:
    parallel_data.append(([tup, possible_targets, f, g, source], {}))

In [52]:
arrangements_iterators[0].cardinality()

360

In [53]:
X.ncpus

13